The following details my work on SDK issue 1860  
https://github.com/AllenInstitute/AllenSDK/issues/1860  

Doug Ollerenshaw
2/9/2021

In [1]:
import pandas as pd
import visual_behavior.database as db
from visual_behavior.translator.foraging2 import data_to_change_detection_core
from multiprocessing import Pool
import os
import json

## paths to PKL files with missing 'omitted_flash_frame_log'

In [2]:
ophys_pkl_paths = [
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_722884882/behavior_session_774937388/774747386.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_722884882/behavior_session_775870895/775749782.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_722884882/behavior_session_777082591/776926613.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_722884882/behavior_session_778980920/778879545.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_722884882/behavior_session_784156661/783998900.pkl',
]

behavior_pkl_paths = [
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_722884882/behavior_session_771245588/181031130535_403491_6cbeebb6-471e-4226-9eec-ae44edab3f27.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_722884882/behavior_session_772125590/181101135909_403491_bc27c196-cad5-41f0-bcf8-e9157db6d2c0.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_722884882/behavior_session_772658979/181102130333_403491_ba848d48-33ea-415e-b9d7-a93dfd1c48de.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_722884882/behavior_session_782380778/181119152241_403491_0991ea0b-9e8a-47cc-b74c-c34f092e2da7.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_722884882/behavior_session_786920476/181128094417_403491_ced6aa12-5730-4f90-a0f7-7ea9f40b4963.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_744911458/behavior_session_771902184/181101092503_412036_9995edab-e40f-49d8-933f-f166cba72d70.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_744911458/behavior_session_772475707/181102095226_412036_565313de-897f-43c5-9127-14d895b9cd0d.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_744911458/behavior_session_773391809/181105091534_412036_c98c7e51-7d51-495e-b950-1b76ba36f5ea.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_744911458/behavior_session_774679623/181107090608_412036_1d9cde8e-c530-4215-b1c4-4ca68679ee17.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_744911458/behavior_session_775291372/181108093606_412036_547f7483-6c42-4027-b001-8db2b4fee2e3.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_744911458/behavior_session_776988190/181112091156_412036_72c43ad0-0dd5-4b0e-b51e-863522466237.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_744911458/behavior_session_778010119/181113084605_412036_740ca61d-b91c-4451-9147-b4b7526d57b0.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_744911458/behavior_session_781938213/181119101741_412036_32300788-81a3-4b78-aa1c-454b3eca5ddd.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_744911458/behavior_session_783217873/181120102547_412036_8132835c-52af-4105-b8f5-e3d4e91fa52c.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_744911458/behavior_session_783959068/181121083447_412036_b6a1162f-ac6a-49c4-96d6-4fa45cdcfd10.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_744911458/behavior_session_785382040/181126092905_412036_c651b112-8ec3-4eb1-b1d4-0e1022b0e64a.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_744911458/behavior_session_786007435/181127100535_412036_bc00be63-a97d-44e7-8f44-13387669f2d8.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_744911458/behavior_session_786938611/181128101549_412036_18f8e587-57bc-4729-973a-56285a8038cb.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_744911458/behavior_session_789008423/181130104604_412036_8201404e-b766-4cbc-94a9-640e329db739.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_744911458/behavior_session_789870329/181203090255_412036_a4adc435-d75c-485b-8e65-d2f913836e29.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_744911458/behavior_session_791248943/181206085911_412036_81a03f75-fd42-49ef-9b8f-8755ae9711f4.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_756577249/behavior_session_772128781/181101150651_416369_6361f115-eedc-4262-be42-4a61ee246b35.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_756577249/behavior_session_772682396/181102141615_416369_cdf57463-7500-4ff7-ac7c-cb3ea6a3cc74.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_756577249/behavior_session_775013029/181107140617_416369_54e6af61-3944-45a2-94d6-9fcb46b8bbf5.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_756577249/behavior_session_775465635/181108132543_416369_7286e519-6963-42aa-8962-a1c12bcf23f4.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_756577249/behavior_session_776123405/181109135224_416369_8bda53e3-4487-42d5-a350-eb803d7d712e.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_756577249/behavior_session_777317267/181112133748_416369_3118831c-505c-4ecb-8cc8-3caab1496ed6.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_756674785/behavior_session_775925580/181109104913_412366_d77375e0-ff0e-46bf-ab4b-2c43444e8b95.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_756674785/behavior_session_777061425/181112103513_412366_6f2f695e-21af-49b8-a350-042214d2e1df.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_756674785/behavior_session_778119826/181113100427_412366_cef5e770-7187-4e08-a540-48ab6029f79b.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_756674785/behavior_session_779580423/181115083459_412366_79db4931-af31-42de-9209-b7d798a96e11.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_756674785/behavior_session_783059206/181120085348_412366_a8a4e775-5b2f-41e0-bc54-6542f51e910f.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_756674785/behavior_session_783971649/181121083853_412366_b9fbf36a-0bc3-455c-8f89-0992fb552ea7.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_756674785/behavior_session_785376297/181126091016_412366_f6d3836d-0580-4b87-9cdb-0f7a05422201.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_756674785/behavior_session_785868918/181127083903_412366_2d553a60-7e46-4781-8cfd-a8d054d5e276.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_756674785/behavior_session_786841415/181128085403_412366_76f204ec-bae5-4572-b623-e866d1523679.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_756674785/behavior_session_788864720/181130091628_412366_6481cbee-edb6-45be-96ad-1b53f9753ddd.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_756674785/behavior_session_789946754/181203102155_412366_d6c7d37d-b0b4-48a4-a3cb-1eb83562aaf7.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_756674785/behavior_session_791278700/181206101439_412366_1a6cb0d4-7f89-4c4e-8f86-10e7243b2bfb.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_760949544/behavior_session_788343533/181129135619_421136_336057e2-8030-4ec8-b608-81d951ae66d6.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_760949544/behavior_session_789221588/181130142903_421136_4e4defe9-f60b-4337-adf3-7f706280683c.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_760949544/behavior_session_791371899/181206141125_421136_ed513968-86c8-43f2-8bdb-9287b5aefbe1.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_760949544/behavior_session_791866254/181207133800_421136_3d574c68-b3ae-4a9e-b78f-5aab6de7b500.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_760949544/behavior_session_792661903/181210132151_421136_85f22fca-68f8-4cd4-8e9e-f2fa8c23644a.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_760949544/behavior_session_793323654/181211125503_421136_be12d700-8aa1-47f3-8703-41d5136955dc.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_760949544/behavior_session_794275479/181212122907_421136_47dfc958-e4f4-4921-b423-9142f717875b.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_760949544/behavior_session_794953909/181213130801_421136_2e062b8e-8847-4cc9-85c1-11212eaac235.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_760949544/behavior_session_795838927/181214143319_421136_b9edb839-5971-46b5-b93a-83962d53d842.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_766015390/behavior_session_790026166/181203143407_423606_9ae1dff9-97a9-4b70-a7dd-9dc64e350290.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_766015390/behavior_session_795838931/181214143152_423606_7b185429-b651-4916-9c7c-d039cb776e4a.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_766015390/behavior_session_797017509/181217133153_423606_a56cd413-fd40-4473-83df-bfa9430a8878.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_766015390/behavior_session_798022584/181218143342_423606_82c363c4-0a2f-4728-8fd1-d501f2363039.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_766015390/behavior_session_799110611/181219141344_423606_6e7ffb1e-c9a2-481b-a418-c4165ea6fa8a.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_766015390/behavior_session_799919873/181220133211_423606_3b93f39c-5959-4d33-bed4-adf618370449.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_766015390/behavior_session_800317148/181221140700_423606_905627a5-55ed-4733-a18b-5e84062d1d25.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_772622653/behavior_session_795854555/181214144601_425496_89b7469f-0b2b-4d09-a0f2-dbde0a063f82.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_772622653/behavior_session_799049628/181219132053_425496_981ab5b5-05aa-4bfd-a0de-93fcc47bc6b2.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_772622653/behavior_session_799919887/181220133144_425496_9e4d19d0-9a7b-4851-b765-8518133d6ce0.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_772622653/behavior_session_800299362/181221133410_425496_d412b09a-43cd-43e6-9fed-ba6c39d33bfe.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_772629813/behavior_session_795854559/181214144752_425493_c80e0ea2-03d5-45fe-afaa-85d0814659df.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_772629813/behavior_session_799051310/181219132217_425493_022e2afa-f8d2-434f-827a-a00643ce442b.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_772629813/behavior_session_799922316/181220133716_425493_adc337eb-d6cc-4fe8-b6d7-9075a880df14.pkl',
    '/allen/programs/braintv/production/visualbehavior/prod0/specimen_772629813/behavior_session_800299366/181221133613_425493_b47445aa-ddf1-4023-b2c9-93563fed1456.pkl',
]

# Start by documenting what we expect to be there

## here is a recent ophys session with omitted flashes

In [3]:
behavior_session_id = 951520319
pkl_path = db.get_pkl_path(behavior_session_id)
data = pd.read_pickle(pkl_path)
core_data = data_to_change_detection_core(data)
print('omitted_flash_probability = {}'.format(core_data['metadata']['params']['flash_omit_probability']))

omitted_flash_probability = 0.05


## here is the location of the omitted flash probability in the PKL file dictionary:

In [4]:
data['items']['behavior']['params']['flash_omit_probability']

0.05

## and here is location/format of the omitted_flash_frame_log

In [5]:
data['items']['behavior']['omitted_flash_frame_log']

{'images_0': [18390,
  21406,
  21541,
  22036,
  22171,
  24331,
  25681,
  27571,
  27661,
  28021,
  28561,
  30496,
  30631,
  31846,
  31936,
  32206,
  36437,
  37157,
  37562,
  43817,
  47598,
  48543,
  48903,
  49038,
  49218,
  52863,
  53088,
  54078,
  55743,
  55923,
  56418,
  56823,
  57409,
  58084,
  58489,
  59299,
  62674,
  63124,
  64160,
  65105,
  65690,
  66815,
  69695,
  70866,
  73071,
  76041,
  76401,
  80271,
  80361,
  89454,
  90174,
  90310,
  96926,
  97556,
  99177,
  101922,
  103723,
  103948,
  106648,
  107189,
  107909,
  108314,
  108809,
  110699,
  112004,
  112139,
  112679,
  114660,
  116145,
  116550,
  117541,
  123212,
  124563,
  126859,
  126949,
  127174,
  129738,
  131538,
  132843,
  133293,
  134148,
  134913,
  136533,
  137253,
  138648,
  139098,
  139278,
  141888,
  145983,
  146658,
  146748,
  148233,
  148413,
  149223,
  150033,
  150168,
  153858,
  156109,
  158404,
  159034,
  160385,
  160475,
  161060,
  162770,
  1

# Here is a recent behavior-only session
This session should not have any omitted flashes - and we get a key-error on `flash_omit_probability`

In [6]:
behavior_session_id = 802364085
pkl_path = db.get_pkl_path(behavior_session_id)
data = pd.read_pickle(pkl_path)
core_data = data_to_change_detection_core(data)
print('omitted_flash_probability = {}'.format(core_data['metadata']['params']['flash_omit_probability']))

KeyError: 'flash_omit_probability'

## also note that the probability key doesn't exist in the PKL file

In [7]:
data['items']['behavior']['params']['flash_omit_probability']

KeyError: 'flash_omit_probability'

## but there is an `'omitted_flash_frame_log'`, which is an empty list

In [8]:
data['items']['behavior']['omitted_flash_frame_log']

{'images_0': []}

# Confirm that all behavior sessions above lack the `'flash_omit_probability'` key, which is an indication that the probability was 0 (i.e., there were no omitted flashes)

In [9]:
def open_pkl_get_flash_omit_probability(pkl_path):
    behavior_session_id = int(pkl_path.split('behavior_session_')[1].split('/')[0])
    data = pd.read_pickle(pkl_path)
    try:
        flash_omit_probability = data['items']['behavior']['params']['flash_omit_probability']
    except KeyError:
        flash_omit_probability = None
        
    return {
        'behavior_session_id':behavior_session_id, 
        'training_stage': data['items']['behavior']['params']['stage'],
        'flash_omit_probability':flash_omit_probability
    }

In [10]:
with Pool(32) as pool:
    behavior_only_omitted_flash_probs = pd.DataFrame(
        pool.map(
            open_pkl_get_flash_omit_probability, 
            behavior_pkl_paths
        )
    )

## There are two sessions in this list that have non-zero omitted flash probabilities, but both are actually ophys sessions!

In [11]:
behavior_only_omitted_flash_probs[pd.notnull(behavior_only_omitted_flash_probs['flash_omit_probability'])]

,behavior_session_id,training_stage,flash_omit_probability
3,782380778,OPHYS_4_images_B,0.05
4,786920476,OPHYS_6_images_B,0.05


# Confirm that all ophys sessions do have an omitted flash probability:

In [12]:
with Pool(32) as pool:
    ophys_omitted_flash_probs = pd.DataFrame(
        pool.map(
            open_pkl_get_flash_omit_probability, 
            ophys_pkl_paths
        )
    )

In [13]:
ophys_omitted_flash_probs

,behavior_session_id,training_stage,flash_omit_probability
0,774937388,OPHYS_1_images_A,0.05
1,775870895,OPHYS_2_images_A_passive,0.05
2,777082591,OPHYS_2_images_A_passive,0.05
3,778980920,OPHYS_3_images_A,0.05
4,784156661,OPHYS_4_images_B,0.05


## confirm that we can identify the frames of the omitted flashes in one of the ophys sessions
Note the warning!

In [14]:
pkl_path = ophys_pkl_paths[0]
data = pd.read_pickle(pkl_path)
core_data = data_to_change_detection_core(data)
core_data['omitted_stimuli'].head()

/allen/programs/braintv/workgroups/nc-ophys/Doug/code/visual_behavior_analysis/visual_behavior/translator/foraging2/extract_stimuli.py:106: UserWarning: 
there was no omitted_flash_frame_log, but long delays were present that were assumed to be omitted flashes
  warnings.warn('\nthere was no omitted_flash_frame_log, but long delays were present that were assumed to be omitted flashes')


,frame,time
0,20060,334.612225
1,21502,358.665696
2,21863,364.687160
3,22268,371.442836
4,23395,390.241960


## write the omitted flashes in the same format that the 'omitted_flash_frame_log' should have been in

In [15]:
{'images_0': core_data['omitted_stimuli']['frame'].to_list()}

{'images_0': [20060,
  21502,
  21863,
  22268,
  23395,
  23350,
  24026,
  25513,
  26864,
  31192,
  32139,
  37368,
  38135,
  41020,
  41517,
  41832,
  42103,
  42779,
  45169,
  49366,
  52744,
  53014,
  53960,
  56257,
  56619,
  59504,
  66400,
  67165,
  67659,
  68019,
  73921,
  74688,
  76584,
  77304,
  77710,
  78386,
  80235,
  82577,
  83794,
  84563,
  87765,
  90830,
  92906,
  92996,
  93131,
  94438,
  94844,
  98813,
  99128,
  100433,
  101380,
  105883,
  106153,
  107552,
  108363,
  109355,
  109580,
  112692,
  112782,
  113369,
  117789,
  120492,
  122568,
  126894,
  127390,
  133202,
  133787,
  134014,
  142708,
  145144,
  149106,
  155460,
  161860,
  162445,
  162625,
  163976,
  165602,
  168801,
  175156,
  176915,
  177410,
  178311,
  188635,
  188995,
  189446,
  198057,
  198326,
  207563,
  218742,
  220048,
  220317,
  221764,
  221719,
  223432,
  224017,
  228751,
  229248]}

# for every session with omitted flashes, save the 'omitted_flash_frame_log' to a file

### define a function

In [19]:
def save_omitted_flash_logs(behavior_session_id):
    pkl_path = db.get_pkl_path(behavior_session_id)
    data = pd.read_pickle(pkl_path)
    core_data = data_to_change_detection_core(data)
    
    savepath = '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/omitted_flash_frame_logs'
    
    filename = os.path.join(
        savepath, 
        'omitted_flash_frame_log_behavior_session_{}.json'.format(behavior_session_id)
    )
    
    omitted_flash_frame_log = {'images_0': core_data['omitted_stimuli']['frame'].to_list()}
    
    with open(filename, "w") as outfile:
        json.dump(
            omitted_flash_frame_log,
            outfile
        )

### get the IDs for which to save files in list form

In [20]:
behavior_session_ids = (
    ophys_omitted_flash_probs['behavior_session_id'].to_list() + 
    behavior_only_omitted_flash_probs[
        pd.notnull(behavior_only_omitted_flash_probs['flash_omit_probability'])
    ]['behavior_session_id'].to_list()
)
behavior_session_ids

[774937388, 775870895, 777082591, 778980920, 784156661, 782380778, 786920476]

### run the function using multiprocessing.Pool

In [21]:
with Pool(32) as pool:
    pool.map(
        save_omitted_flash_logs, 
        behavior_session_ids
    )

/allen/programs/braintv/workgroups/nc-ophys/Doug/code/visual_behavior_analysis/visual_behavior/translator/foraging2/extract_stimuli.py:106: UserWarning: 
there was no omitted_flash_frame_log, but long delays were present that were assumed to be omitted flashes
  warnings.warn('\nthere was no omitted_flash_frame_log, but long delays were present that were assumed to be omitted flashes')
/allen/programs/braintv/workgroups/nc-ophys/Doug/code/visual_behavior_analysis/visual_behavior/translator/foraging2/extract_stimuli.py:106: UserWarning: 
there was no omitted_flash_frame_log, but long delays were present that were assumed to be omitted flashes
  warnings.warn('\nthere was no omitted_flash_frame_log, but long delays were present that were assumed to be omitted flashes')
/allen/programs/braintv/workgroups/nc-ophys/Doug/code/visual_behavior_analysis/visual_behavior/translator/foraging2/extract_stimuli.py:106: UserWarning: 
there was no omitted_flash_frame_log, but long delays were present t